In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

import xgboost as xgb
from xgboost import plot_importance, plot_tree

import utility_functions as fn

In [2]:
energy = pd.read_csv('energy_clean.csv',index_col=0)
energy.index = pd.to_datetime(energy.index,format='%Y-%m-%d %H:00:00')

energy['Yale_kW']= energy.iloc[:, 0:11].sum(axis=1)
energy = pd.DataFrame(energy['Yale_kW'])

In [3]:
weather = pd.read_csv('weather_clean.csv',index_col=0)
weather.index = pd.to_datetime(weather.index,format='%Y-%m-%d %H:00:00')

weather = fn.add_hours_before(weather,np.arange(1,24))
weather = weather.loc[energy.index[0]:energy.index[len(energy)-1]]

In [12]:
# dates = pd.read_csv('date_features.csv',index_col=0)
# dates.index = pd.to_datetime(dates.index,format='%Y-%m-%d %H:00:00')
# dates = dates.loc[energy.index[0]:energy.index[len(energy)-1]]
dates = fn.date_features(energy)

DatetimeIndex(['2018-01-01 01:00:00', '2018-01-01 02:00:00',
               '2018-01-01 03:00:00', '2018-01-01 04:00:00',
               '2018-01-01 05:00:00', '2018-01-01 06:00:00',
               '2018-01-01 07:00:00', '2018-01-01 08:00:00',
               '2018-01-01 09:00:00', '2018-01-01 10:00:00',
               ...
               '2018-07-27 14:00:00', '2018-07-27 15:00:00',
               '2018-07-27 16:00:00', '2018-07-27 17:00:00',
               '2018-07-27 18:00:00', '2018-07-27 19:00:00',
               '2018-07-27 20:00:00', '2018-07-27 21:00:00',
               '2018-07-27 22:00:00', '2018-07-27 23:00:00'],
              dtype='datetime64[ns]', length=4991, freq=None)

In [5]:
split_date = '2018-07-26 00:00:00'

weather_train = weather.loc[weather.index <= split_date].copy()
weather_test = weather.loc[weather.index > split_date].copy()

dates_train = dates.loc[dates.index <= split_date].copy()
dates_test = dates.loc[dates.index > split_date].copy()

In [6]:
X_train = weather_train.join(dates_train)
X_test = weather_test.join(dates_test)

y_train = energy.loc[energy.index <= split_date].copy()
y_test = energy.loc[energy.index > split_date].copy()

In [8]:
reg = xgb.XGBRegressor(n_estimators=1000)
reg.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        early_stopping_rounds=50,
        verbose=False)

ValueError: DataFrame for label cannot have multiple columns

In [ ]:
_ = plot_importance(reg, height=0.9)

In [ ]:
y_test['Prediction_kW'] = reg.predict(X_test)
yale_all = pd.concat([y_test, y_train], sort=False)
_ = yale_all[['Yale_kW','Prediction_kW']].plot(figsize=(15, 5))

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    """Calculates MAPE given y_true and y_pred"""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
mean_absolute_percentage_error(y_true=y_test['Yale_kW'], y_pred=y_test['Prediction_kW'])

In [ ]:
y_test